# BasicTick: Update HDB's Database Version
This noteboook will update the HDB cluster to use the most recent version (changeset) of the database to reflect a change created by the RDB.

In [1]:
# clusters
RDB_CLUSTER_NAME="RDB_basictickdb_20230601"
HDB_CLUSTER_NAME="HDB_basictickdb_20230601"

# database
DB_NAME="basictickdb"

In [2]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env2 import *


In [3]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


# Cluster Details

In [4]:
try:
    resp=client.get_kx_cluster(environmentId=ENV_ID, clusterName=HDB_CLUSTER_NAME)
    resp.pop('ResponseMetadata')
except client.exceptions.ResourceNotFoundException:
    resp=f"Cluster: {HDB_CLUSTER_NAME} Not Found"

display(resp)

{'status': 'RUNNING',
 'clusterName': 'HDB_basictickdb_20230601',
 'clusterType': 'HDB',
 'databases': [{'databaseName': 'basictickdb',
   'cacheConfigurations': [{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}],
   'changesetId': 'uMQ5qhWLaIRqY97nyAU9cg'}],
 'cacheStorageConfigurations': [{'type': 'CACHE_1000', 'size': 1200}],
 'clusterDescription': 'Created with create_HDB for basic_tick notebook',
 'capacityConfiguration': {'nodeType': 'kx.s.xlarge', 'nodeCount': 3},
 'releaseLabel': '1.0',
 'vpcConfiguration': {'vpcId': 'vpc-0e702dec545865b11',
  'securityGroupIds': ['sg-018111774e795682d'],
  'subnetIds': ['subnet-0f97cae6600859c17'],
  'ipAddressType': 'IP_V4'},
 'initializationScript': 'basictick/hdbmkdb.q',
 'commandLineArguments': [{'key': 's', 'value': '8'},
  {'key': 'dbname', 'value': 'basictickdb'},
  {'key': 'codebase', 'value': 'basictick'}],
 'code': {'s3Bucket': 'kdb-demo-612841383594', 's3Key': 'code/basictick.zip'},
 'lastModifiedTimestamp': datetime.datetime(2023, 6, 

# Database Details

In [5]:
dump_database(client, ENV_ID, DB_NAME)

Database: basictickdb, Changesets: 
Welcome kdb database
Bytes: 128,321,636 Changesets: 4 Files: 49
----------------------------------------------------------------------------------------------------


,changesetId,createdTimestamp,activeFromTimestamp,lastModifiedTimestamp,status
0,6MQ7xgjt29JRIm3iuzFPxg,2023-06-01 15:20:48.348000+00:00,2023-06-01 15:21:27.636000+00:00,2023-06-01 15:20:48.348000+00:00,COMPLETED
1,uMQ5qhWLaIRqY97nyAU9cg,2023-05-31 19:41:15.927000+00:00,2023-05-31 19:41:32.752000+00:00,2023-05-31 19:41:15.928000+00:00,COMPLETED
2,QsQ3lbQfnmumBgZOeiPIOw,2023-05-31 00:18:15.744000+00:00,2023-05-31 00:18:48.111000+00:00,2023-05-31 00:18:15.744000+00:00,COMPLETED
3,ZsQ3eXNny4UaexiXtmEkZg,2023-05-30 23:22:00.273000+00:00,2023-05-30 23:22:26.348000+00:00,2023-05-30 23:22:00.273000+00:00,COMPLETED


In [6]:
db = client.get_kx_database(environmentId=ENV_ID, databaseName=DB_NAME)

try:
    c_set_list = client.list_kx_changesets(environmentId=ENV_ID, databaseName=DB_NAME)['kxChangesets']
except:
    note_str = "<<Could not get changesets>>"

# sort by create time, most recent first
c_set_list = sorted(c_set_list, key=lambda d: d['createdTimestamp'], reverse=True) 

# first changeset
CHANGESET_ID=c_set_list[0]['changesetId']

print(f"Latest Changeset")
display(c_set_list[0])
print()

print(f"ChangesetId: {CHANGESET_ID}")
print()
c_set = client.get_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, changesetId=CHANGESET_ID)
display(c_set)

Latest Changeset


{'changesetId': '6MQ7xgjt29JRIm3iuzFPxg',
 'createdTimestamp': datetime.datetime(2023, 6, 1, 15, 20, 48, 348000, tzinfo=tzlocal()),
 'activeFromTimestamp': datetime.datetime(2023, 6, 1, 15, 21, 27, 636000, tzinfo=tzlocal()),
 'lastModifiedTimestamp': datetime.datetime(2023, 6, 1, 15, 20, 48, 348000, tzinfo=tzlocal()),
 'status': 'COMPLETED'}


ChangesetId: 6MQ7xgjt29JRIm3iuzFPxg



{'ResponseMetadata': {'RequestId': '57cd0583-1e81-465c-a62a-a5e767ee95bd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'content-length': '544',
   'connection': 'keep-alive',
   'date': 'Thu, 01 Jun 2023 15:22:00 GMT',
   'x-amzn-requestid': '57cd0583-1e81-465c-a62a-a5e767ee95bd',
   'x-amz-apigw-id': 'F2Gf2G12IAMF-CQ=',
   'x-amzn-trace-id': 'Root=1-6478b798-15066df237391d4a4845b065',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 aef197034a978e986954f2826c90b090.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 'HOc0kTlhJO60KIX4iEN02nsQX3DJjT-7Avukpw9hVJ2bN3cilwrU_Q=='},
  'RetryAttempts': 0},
 'changesetId': '6MQ7xgjt29JRIm3iuzFPxg',
 'databaseName': 'basictickdb',
 'environmentId': 'itcdoumzc5cixt5vh4t6dp',
 'changeRequests': [{'changeType': 'PUT',
   's3Path': 's3://finspace-landing-us-east-1-itcdoumzc5cixt5vh4t6dp/jgYOL_krL4tiTGXIFfEtV/0/',
   'dbPath': '/2023.06.01/'},
  {'changeType': 'PUT',
   '

## Update Cluster with latest changeset

In [7]:
DB_CONFIG=[{'databaseName': DB_NAME,
   'cacheConfigurations': [{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}],
   'changesetId': CHANGESET_ID}]

resp=client.update_kx_cluster_databases(environmentId=ENV_ID, clusterName=HDB_CLUSTER_NAME, databases=DB_CONFIG)
resp

{'ResponseMetadata': {'RequestId': '879fb7ef-837f-4fe3-80de-12dc1e76908a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'date': 'Thu, 01 Jun 2023 15:22:03 GMT',
   'x-amzn-requestid': '879fb7ef-837f-4fe3-80de-12dc1e76908a',
   'x-amz-apigw-id': 'F2Gf5E26IAMFdnQ=',
   'x-amzn-trace-id': 'Root=1-6478b798-31bb321510e96ea31626f588',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 aef197034a978e986954f2826c90b090.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 'hzTk7MpKuqgQiObTfBAOOwMzwtidVt093UG4xAns1R6KQznNp4rmrw=='},
  'RetryAttempts': 0}}

In [8]:
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=HDB_CLUSTER_NAME, show_wait=True)

Cluster: HDB_basictickdb_20230601 status is UPDATING, total wait 0:00:00, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is UPDATING, total wait 0:00:30, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is UPDATING, total wait 0:01:00, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is UPDATING, total wait 0:01:30, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is UPDATING, total wait 0:02:00, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is UPDATING, total wait 0:02:30, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is UPDATING, total wait 0:03:00, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is UPDATING, total wait 0:03:30, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is UPDATING, total wait 0:04:00, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is UPDATING, total wait 0:04:30, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is UPDATING, total wait 0:05:

{'status': 'RUNNING',
 'clusterName': 'HDB_basictickdb_20230601',
 'clusterType': 'HDB',
 'databases': [{'databaseName': 'basictickdb',
   'cacheConfigurations': [{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}],
   'changesetId': '6MQ7xgjt29JRIm3iuzFPxg'}],
 'cacheStorageConfigurations': [{'type': 'CACHE_1000', 'size': 1200}],
 'clusterDescription': 'Created with create_HDB for basic_tick notebook',
 'capacityConfiguration': {'nodeType': 'kx.s.xlarge', 'nodeCount': 3},
 'releaseLabel': '1.0',
 'vpcConfiguration': {'vpcId': 'vpc-0e702dec545865b11',
  'securityGroupIds': ['sg-018111774e795682d'],
  'subnetIds': ['subnet-0f97cae6600859c17'],
  'ipAddressType': 'IP_V4'},
 'initializationScript': 'basictick/hdbmkdb.q',
 'commandLineArguments': [{'key': 's', 'value': '8'},
  {'key': 'dbname', 'value': 'basictickdb'},
  {'key': 'codebase', 'value': 'basictick'}],
 'code': {'s3Bucket': 'kdb-demo-612841383594', 's3Key': 'code/basictick.zip'},
 'lastModifiedTimestamp': datetime.datetime(2023, 6, 

# Update the Gateway
reinit the gateway to reacquire the secure connection string the HDB now that its back.

# Show Updated HDB
You can now return to the create_EOD_changset or query_HDB_updated notebooks to query the HDB after the update.

- [create_EOD_changeset](create_EOD_changeset.ipynb)
- [query_HDB_updated](query_HDB_updated.ipynb)

In [9]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2023-06-01 15:36:45.722074
